In [9]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import radon, iradon
from scipy.io import loadmat
from scipy.stats import poisson
from skimage.metrics import mean_squared_error as mse
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import cv2
# from skimage.metrics import mean_squared_error as mse
# from skimage.metrics import peak_signal_noise_ratio as psnr
# from skimage.metrics import structural_similarity as ssim

import scienceplots

plt.style.use(['science', 'notebook', 'grid'])

In [37]:
def ART(A, AT, b, x, mu=1e0, niter=1e2, bpos=True, plot=False):

    ATA = AT(A(np.ones_like(x)))  # Imagen a partir del sinograma de una imagen de unos  

    for i in range(int(niter)):

        x = x + np.divide(mu * AT(b - A(x)), ATA)

        if bpos:
            x[x < 0] = 0

        if plot:
            plt.imshow(x, cmap='gray')
            plt.title("%d / %d" % (i + 1, niter))
            plt.pause(1)
            plt.close()

    return x

# def sim_ART(A, AT, b, x, mu=1e0, niter=1e2, bpos=True, plot=False):

#     ATA = AT(A(np.ones_like(x)))  # Imagen a partir del sinograma de una imagen de unos  

#     for i in range(int(niter)):
#         # Actualiza de manera directa
#         x = x + np.divide(mu * AT(b - A(x)), ATA)

#         # Actualizo de manera inversa
#         x = x + np.divide(mu * AT(b[:, ::-1] - A(x)[:, ::-1]), ATA)

#         if bpos:
#             x[x < 0] = 0

#         if plot:
#             plt.imshow(x, cmap='gray')
#             plt.title("%d / %d" % (i + 1, niter))
#             plt.pause(1)
#             plt.close()

#     return x

def sim_art(A, AT, b, x, mu=1e0, niter=1e2, bpos=True, plot=False):
    ATA = AT(A(np.ones_like(x)))  # Image from the sinogram of an image of ones  

    num_projections = b.shape[1]

    for i in range(int(niter)):
        # Generate a random permutation of the projection indices
        indices = np.arange(0, num_projections, 1, dtype=np.int64)

        if i % 2 != 0:
            indices = indices[::-1]

        # Update using the randomly selected projections
        for j in indices:
            # Create a copy of b with all projections set to zero
            b_zeroed = np.zeros_like(b)
            # Set the current projection in b_zeroed to the corresponding projection in b
            b_zeroed[:, j] = b[:, j]
            # Update x using b_zeroed and the full x
            x = x + np.divide(mu * AT(b_zeroed - A(x)), ATA)

        if bpos:
            x[x < 0] = 0

        if plot:
            plt.imshow(x, cmap='gray')
            plt.title("%d / %d" % (i + 1, niter))
            plt.pause(1)
            plt.close()


In [33]:
# generate and array that contains the indices of the projections, and the inverse



array([], dtype=int64)

In [39]:
## SYSTEM SETTING
N = 512
ANG = 180
VIEW = 360
THETA = np.linspace(0, ANG, VIEW + 1)
THETA = THETA[:-1]

A = lambda x: radon(x, THETA, circle=False).astype(np.float32)  # Transforma en Radon
AT = lambda y: iradon(y, THETA, circle=False, filter_name=None, output_size=N).astype(np.float32)/(np.pi/(2*len(THETA))) # Transforma en Radon inverso y normaliza por pi/2Ntheta, sin filtro
AINV = lambda y: iradon(y, THETA, circle=False, output_size=N).astype(np.float32)   # Transforma en Radon inverso sin normalizar y con un filtro de rampa


## DATA GENERATION
x = loadmat('XCAT512.mat')['XCAT512']   # matriz de 512x512 con la imagen original
p = A(x)    # Sinograma de la imagen original, fila detectores y columna angulos
x_full = AINV(p)    # Imagen reconstruida con el sinograma p
N_iter = 10

x_sim = sim_art(A, AT, p, np.zeros_like(x), mu=1e-1, niter=N_iter, bpos=True, plot=False)
x_art = ART(A, AT, p, np.zeros_like(x), mu=1e-1, niter=N_iter, bpos=True, plot=False)

print('MSE ART:', mse(x, x_art))
print('MSE SIM:', mse(x, x_sim))
print('PSNR ART:', psnr(x, x_art, data_range=1))
print('PSNR SIM:', psnr(x, x_sim, data_range=1))
print('SSIM ART:', ssim(x, x_art, data_range=1))
print('SSIM SIM:', ssim(x, x_sim, data_range=1))

plt.figure(figsize=(10, 10))
plt.subplot(221)
plt.imshow(x, cmap='gray')
plt.title('Imagen original')
plt.subplot(222)
plt.imshow(p, cmap='gray')
plt.title('Sinograma')
plt.subplot(223)
plt.imshow(x_art, cmap='gray')
plt.title('Imagen ART')
plt.subplot(224)
plt.imshow(x_sim, cmap='gray')
plt.title('Imagen SIM')
plt.show()


KeyboardInterrupt: 

In [ ]:
## SYSTEM SETTING
N = 512
ANG = 180
VIEW = 360
THETA = np.linspace(0, ANG, VIEW + 1)
THETA = THETA[:-1]

A = lambda x: radon(x, THETA, circle=False).astype(np.float32)  # Transforma en Radon
AT = lambda y: iradon(y, THETA, circle=False, filter_name=None, output_size=N).astype(np.float32)/(np.pi/(2*len(THETA))) # Transforma en Radon inverso y normaliza por pi/2Ntheta, sin filtro
AINV = lambda y: iradon(y, THETA, circle=False, output_size=N).astype(np.float32)   # Transforma en Radon inverso sin normalizar y con un filtro de rampa


## DATA GENERATION
x = loadmat('XCAT512.mat')['XCAT512']   # matriz de 512x512 con la imagen original
p = A(x)    # Sinograma de la imagen original
x_full = AINV(p)    # Imagen reconstruida con el sinograma p

## LOW-DOSE SINOGRAM GENERATION
i0 = 5e4
pn = np.exp(-p)
pn = i0*pn
pn = poisson.rvs(pn)
pn[pn < 1] = 1
pn = -np.log(pn/i0)
pn[pn < 0] = 0  # clipeo los valores negativos

y = pn  # Sinograma de la imagen

# plt.imshow(y, cmap='gray', aspect='auto')

## Algebraic Reconstruction Technique (ART) INITIALIZATION
x_low = AINV(y)
x0 = np.zeros_like(x)
mu = 1e0
niter = 5
bpos = True

x_art = ART(A, AT, y, x0, mu, niter, bpos)

## CALCULATE QUANTIFICATION FACTOR
x_low[x_low < 0] = 0
x_art[x_art < 0] = 0
nor = np.amax(x)

mse_x_low = mse(x/nor, x_low/nor)
mse_x_art = mse(x/nor, x_art/nor)

data_range = max((x/nor).max(), (x_low/nor).max(), (x_art/nor).max()) - min((x/nor).min(), (x_low/nor).min(), (x_art/nor).min())

psnr_x_low = psnr(x/nor, x_low/nor, data_range=data_range)
psnr_x_art = psnr(x/nor, x_art/nor, data_range=data_range)

ssim_x_low = ssim(x_low/nor, x/nor, data_range=data_range)
ssim_x_art = ssim(x_art/nor, x/nor, data_range=data_range)

## DISPLAY
wndImg = [0, 0.03]
wndPrj = [0, 6]

plt.subplot(241)
plt.imshow(x, cmap='gray', vmin=wndImg[0], vmax=wndImg[1])
plt.axis('off')
plt.axis('image')
plt.title('Ground truth')

plt.subplot(242)
plt.imshow(x_full, cmap='gray', vmin=wndImg[0], vmax=wndImg[1])
plt.axis('off')
plt.axis('image')
plt.title('full-dose')

plt.subplot(243)
plt.imshow(x_low, cmap='gray', vmin=wndImg[0], vmax=wndImg[1])
plt.axis('off')
plt.axis('image')
plt.title('low-dose\nMSE: %.4f\nPSNR: %.4f\nSSIM: %.4f' % (mse_x_low, psnr_x_low, ssim_x_low))

plt.subplot(244)
plt.imshow(x_art, cmap='gray', vmin=wndImg[0], vmax=wndImg[1])
plt.axis('off')
plt.axis('image')
plt.title('ART\nMSE: %.4f\nPSNR: %.4f\nSSIM: %.4f' % (mse_x_art, psnr_x_art, ssim_x_art))

plt.subplot(246)
plt.imshow(p, cmap='gray', vmin=wndPrj[0], vmax=wndPrj[1])
plt.title('full-dose\n(VIEW: %d)' % VIEW)
plt.xlabel('View')
plt.ylabel('Detector')

plt.subplot(247)
plt.imshow(y, cmap='gray', vmin=wndPrj[0], vmax=wndPrj[1])
plt.title('low-dose\n(VIEW: %d)' % VIEW)
plt.xlabel('View')
plt.ylabel('Detector')

plt.subplot(248)
plt.imshow(y - p, cmap='gray')
plt.title('full-dose - low-dose\n(Poisson noise)')
plt.xlabel('View')
plt.ylabel('Detector')

plt.show()
